# User–Project–Quarter Panel (Refactored)

Refactored pipeline to construct a user×project×quarter panel with:
- commit-level collapse
- counts and AI means
- library/pair/combo novelty metrics
- community enrichments (PMI‑Louvain, RCA‑SBM)
- cushion and year subset

Use `run_pipeline_in_notebook(...)` at the end to execute on your CSVs.


In [ ]:
from __future__ import annotations

from dataclasses import dataclass
from typing import List, Optional, Set, Tuple, Dict, Any
import ast
import itertools as it

import numpy as np
import pandas as pd


In [ ]:
# ---------------------------------------
# Settings
# ---------------------------------------
@dataclass(frozen=True)
class Settings:
    cushion_quarters: int = 3               # drop first K quarters per user
    global_year_min: Optional[int] = 2020   # keep rows with year > this value; None=skip
    top_k_libs: Optional[int] = 1000        # None => choose by coverage
    top_k_coverage: float = 0.80            # used only if top_k_libs is None
    min_combo_size: int = 2                 # commit combo size threshold


In [ ]:
# ---------------------------------------
# Utilities
# ---------------------------------------
def _safe_literal_eval(x: Any) -> List[Any]:
    """Safely parse a stringified Python literal into a Python object. Returns []."""
    if x is None or (isinstance(x, float) and np.isnan(x)):
        return []
    if isinstance(x, (list, tuple, set)):
        return list(x)
    s = str(x).strip()
    if not s or s == '[]':
        return []
    try:
        val = ast.literal_eval(s)
        if isinstance(val, (list, tuple, set)):
            return list(val)
        return [val]
    except Exception:
        return []


def _standardize_yq(series: pd.Series) -> pd.Series:
    """Normalize 'year_quarter' variants to 'YYYYQx'."""
    def _norm(yq: Any) -> str:
        if yq is None or (isinstance(yq, float) and np.isnan(yq)):
            return ''
        s = str(yq).strip().upper().replace('-', '')
        year = ''.join(ch for ch in s if ch.isdigit())[:4]
        qpos = s.find('Q')
        if year and qpos != -1 and qpos + 1 < len(s) and s[qpos+1].isdigit():
            return f"{year}Q{s[qpos+1]}"
        last_digit = next((ch for ch in reversed(s) if ch.isdigit()), '')
        return f"{year}Q{last_digit}" if year and last_digit else s

    return series.astype(str).map(_norm)


def _quarter_order_map(yq: pd.Series) -> Dict[str, int]:
    uniq = pd.Index(sorted(pd.Index(yq.unique())))
    return {v: i for i, v in enumerate(uniq)}


def _ordered_by_quarter(df: pd.DataFrame, yq_col: str = 'year_quarter') -> pd.DataFrame:
    order = _quarter_order_map(df[yq_col])
    return df.assign(_ord=df[yq_col].map(order)).sort_values(['_ord'])


In [ ]:
# ---------------------------------------
# Commit-level preparation
# ---------------------------------------
def build_commit_level(df: pd.DataFrame) -> pd.DataFrame:
    """Collapse function/file-level rows to one row per commit_id."""
    required = ['user_hashed','project','year_quarter','commit_id','file_name','imports_commit']
    missing = [c for c in required if c not in df.columns]
    if missing:
        raise KeyError(f"Missing columns in raw df: {missing}")

    df = df.copy()
    df['year_quarter'] = _standardize_yq(df['year_quarter'])
    df['_libs_row'] = df['imports_commit'].apply(_safe_literal_eval)

    commit_file = df[['user_hashed','project','year_quarter','commit_id','file_name']].drop_duplicates()
    files_per_commit = (
        commit_file
        .groupby(['user_hashed','project','year_quarter','commit_id'], as_index=False)
        .agg(n_files=('file_name', 'nunique'))
    )

    libs_per_commit = (
        df.groupby(['user_hashed','project','year_quarter','commit_id'], as_index=False)
          .agg(libs=('_libs_row', lambda L: sorted(set(str(z).strip() for sub in L for z in (sub if isinstance(sub, (list, tuple, set)) else [sub]) if str(z).strip()))))
    )
    libs_per_commit['import_commit'] = libs_per_commit['libs'].apply(lambda L: int(len(L) > 0))

    commit_level = files_per_commit.merge(libs_per_commit, on=['user_hashed','project','year_quarter','commit_id'], how='outer')
    return commit_level[['commit_id','user_hashed','project','year_quarter','n_files','import_commit','libs']]


In [ ]:
# ---------------------------------------
# Base panel
# ---------------------------------------
def build_base_panel(df: pd.DataFrame, commit_level: pd.DataFrame) -> pd.DataFrame:
    # counts per (u,p,yq)
    counts = (
        commit_level
        .groupby(['user_hashed','project','year_quarter'], as_index=False)
        .agg(
            n_commits=('commit_id','nunique'),
            n_commits_multi_files=('n_files', lambda s: int((s > 1).sum())),
            n_commits_with_import=('import_commit','sum')
        )
    )

    # user-quarter AI means
    ai_cols = [c for c in ('ai_share','ai_share_window') if c in df.columns]
    ai_agg = {}
    if 'ai_share' in ai_cols:
        ai_agg['mean_ai_share'] = ('ai_share','mean')
    if 'ai_share_window' in ai_cols:
        ai_agg['ai_share_window'] = ('ai_share_window','mean')
    ai_user_yq = df.groupby(['user_hashed','year_quarter'], as_index=False).agg(**ai_agg) if ai_agg else pd.DataFrame(columns=['user_hashed','year_quarter'])

    # full grid
    user_project_pairs = df[['user_hashed','project']].drop_duplicates()
    all_yq = pd.DataFrame({'year_quarter': np.sort(df['year_quarter'].unique())})
    full_grid = user_project_pairs.assign(_k=1).merge(all_yq.assign(_k=1), on='_k', how='outer').drop(columns=['_k'])

    panel = full_grid.merge(counts, on=['user_hashed','project','year_quarter'], how='left')                      .merge(ai_user_yq, on=['user_hashed','year_quarter'], how='left')

    panel[['n_commits','n_commits_multi_files','n_commits_with_import']] = (
        panel[['n_commits','n_commits_multi_files','n_commits_with_import']].fillna(0).astype('int64')
    )

    # project size summaries
    project_sizes = (
        df.groupby('project', as_index=False)
          .agg(
              project_n_people=('user_hashed','nunique'),
              project_n_commits=('commit_id','nunique'),
              project_n_files=('file_name','nunique')
          )
    )
    panel = panel.merge(project_sizes, on='project', how='left')

    # restrict to between first & last active quarter for each (user, project)
    order = _quarter_order_map(df['year_quarter'])
    panel['_ord'] = panel['year_quarter'].map(order)
    bounds = (
        panel.loc[panel['n_commits'] > 0, ['user_hashed','project','_ord']]
             .groupby(['user_hashed','project'], as_index=False)
             .agg(first_ord=('_ord','min'), last_ord=('_ord','max'))
    )
    panel = panel.merge(bounds, on=['user_hashed','project'], how='left')
    panel = panel[(panel['_ord'] >= panel['first_ord']) & (panel['_ord'] <= panel['last_ord'])]                 .drop(columns=['_ord','first_ord','last_ord'])                 .reset_index(drop=True)

    panel['year'] = panel['year_quarter'].str.slice(0,4).astype('int64')
    panel['quarter'] = panel['year_quarter'].str.slice(-1)
    return panel


In [ ]:
# ---------------------------------------
# Novelty helpers
# ---------------------------------------
def _per_user_chronological_counts(df: pd.DataFrame, list_col: str, user_col: str='user_hashed', yq_col: str='year_quarter'):
    uniq_counts: List[int] = []
    new_counts: List[int] = []
    for _, g in df.groupby(user_col, sort=False):
        seen: Set[Any] = set()
        for items in g[list_col]:
            cur = set(items)
            uniq_counts.append(len(cur))
            new_counts.append(len(cur - seen))
            seen |= cur
    return uniq_counts, new_counts


def _global_novelty_over_time(df: pd.DataFrame, list_col: str, yq_col: str='year_quarter', order_map: Optional[Dict[str,int]] = None) -> List[int]:
    if order_map is None:
        order_map = _quarter_order_map(df[yq_col])
    tmp = df.assign(_ord=df[yq_col].map(order_map)).sort_values(['_ord','user_hashed'])
    seen: Set[Any] = set()
    out: List[int] = []
    for items in tmp[list_col]:
        cur = set(items)
        out.append(len(cur - seen))
        seen |= cur
    tmp = tmp.assign(_glob=out)
    tmp = tmp[['user_hashed','year_quarter','_glob']].reset_index(drop=True)
    return df.merge(tmp, on=['user_hashed','year_quarter'], how='left')['_glob'].tolist()


def commit_libs(df_commit: pd.DataFrame) -> pd.DataFrame:
    cols = ['commit_id','user_hashed','year_quarter','libs']
    missing = [c for c in cols if c not in df_commit.columns]
    if missing:
        raise KeyError(f"Missing columns in commit_level for commit_libs: {missing}")
    return df_commit[cols].copy()


In [ ]:
# ---------------------------------------
# Library metrics (user×quarter)
# ---------------------------------------
def build_user_quarter_library_metrics(panel: pd.DataFrame, df_commit: pd.DataFrame) -> pd.DataFrame:
    cl = commit_libs(df_commit)
    uq = (
        cl.groupby(['user_hashed','year_quarter'], as_index=False)
          .agg(lib_lists=('libs', list))
          .sort_values(['user_hashed','year_quarter'])
    )
    uq['libs_flat'] = uq['lib_lists'].apply(lambda lists: [lib for row in lists for lib in row])

    uniq, new = _per_user_chronological_counts(uq, 'libs_flat')
    uq['unique_libs'] = uniq
    uq['new_unique_libs'] = new
    uq['globally_new_libs'] = _global_novelty_over_time(uq, 'libs_flat')

    grid = panel[['user_hashed','year_quarter']].drop_duplicates()
    out = grid.merge(uq[['user_hashed','year_quarter','unique_libs','new_unique_libs','globally_new_libs']],
                     on=['user_hashed','year_quarter'], how='left')
    out[['unique_libs','new_unique_libs','globally_new_libs']] = out[['unique_libs','new_unique_libs','globally_new_libs']].fillna(0).astype('int64')
    return out


In [ ]:
# ---------------------------------------
# Pair metrics (top‑K libraries)
# ---------------------------------------
def build_user_quarter_pair_metrics(panel: pd.DataFrame, df_commit: pd.DataFrame, top_k_libs: Optional[int]=None, top_k_coverage: float=0.80) -> pd.DataFrame:
    cl = commit_libs(df_commit)
    lib_freq = pd.Series((lib for libs in cl['libs'] for lib in libs)).value_counts()
    if lib_freq.empty:
        k_libs: Set[str] = set()
    else:
        if top_k_libs is None:
            cum = lib_freq.cumsum() / lib_freq.sum()
            k = int((cum <= top_k_coverage).sum())
            k = max(k, 1)
        else:
            k = int(top_k_libs)
        k_libs = set(lib_freq.index[:k].tolist())

    cl = cl.assign(libs_topk=cl['libs'].apply(lambda L: sorted(set(l for l in L if l in k_libs))))
    cl['pairs'] = cl['libs_topk'].apply(lambda s: [tuple(sorted(p)) for p in it.combinations(s, 2)] if len(s) >= 2 else [])

    uq = (
        cl.groupby(['user_hashed','year_quarter'], as_index=False)
          .agg(pair_lists=('pairs', list))
          .sort_values(['user_hashed','year_quarter'])
    )
    uq['pairs_flat'] = uq['pair_lists'].apply(lambda lists: [p for row in lists for p in row])

    uniq, new = _per_user_chronological_counts(uq, 'pairs_flat')
    uq['unique_pairs_topk'] = uniq
    uq['new_unique_pairs_topk'] = new
    uq['globally_new_pairs_topk'] = _global_novelty_over_time(uq, 'pairs_flat')

    grid = panel[['user_hashed','year_quarter']].drop_duplicates()
    out = grid.merge(uq[['user_hashed','year_quarter','unique_pairs_topk','new_unique_pairs_topk','globally_new_pairs_topk']],
                     on=['user_hashed','year_quarter'], how='left')
    cols = ['unique_pairs_topk','new_unique_pairs_topk','globally_new_pairs_topk']
    out[cols] = out[cols].fillna(0).astype('int64')
    return out


In [ ]:
# ---------------------------------------
# Commit combo metrics (≥ min size)
# ---------------------------------------
def build_user_quarter_commit_combo_metrics(panel: pd.DataFrame, df_commit: pd.DataFrame, min_combo_size: int=2) -> pd.DataFrame:
    cl = commit_libs(df_commit)
    cl['combo'] = cl['libs'].apply(lambda s: tuple(sorted(set(s))) if len(set(s)) >= min_combo_size else None)
    cl = cl[cl['combo'].notna()].copy()

    uq = (
        cl.groupby(['user_hashed','year_quarter'], as_index=False)
          .agg(combo_list=('combo', list))
          .sort_values(['user_hashed','year_quarter'])
    )

    uniq, new = _per_user_chronological_counts(uq, 'combo_list')
    uq['unique_commit_combos'] = uniq
    uq['new_unique_commit_combos'] = new
    uq['globally_new_commit_combos'] = _global_novelty_over_time(uq, 'combo_list')

    grid = panel[['user_hashed','year_quarter']].drop_duplicates()
    out = grid.merge(uq[['user_hashed','year_quarter','unique_commit_combos','new_unique_commit_combos','globally_new_commit_combos']],
                     on=['user_hashed','year_quarter'], how='left')
    cols = ['unique_commit_combos','new_unique_commit_combos','globally_new_commit_combos']
    out[cols] = out[cols].fillna(0).astype('int64')
    return out


In [ ]:
# ---------------------------------------
# Community enrichment
# ---------------------------------------
def _build_library_to_community_map(frame: pd.DataFrame, list_col: str) -> Dict[str, Any]:
    m = frame[['community', list_col]].copy()
    m[list_col] = m[list_col].apply(_safe_literal_eval)
    m = m.explode(list_col, ignore_index=True)
    m['library'] = m[list_col].astype(str).str.strip()
    m = m.dropna(subset=['library']).drop_duplicates('library', keep='first')
    return dict(m[['library','community']].values)


def _build_project_to_community(frame: pd.DataFrame) -> pd.DataFrame:
    m = frame[['community','description_simple','projects']].copy()
    m['projects'] = m['projects'].apply(_safe_literal_eval)
    m = m.explode('projects', ignore_index=True)
    m = m.rename(columns={'projects':'project'})
    m['project'] = m['project'].astype(str).str.strip()
    m = m.drop_duplicates(subset='project', keep='first')
    return m[['project','community','description_simple']].copy()


def attach_project_communities(panel: pd.DataFrame,
                               cluster_pmi_louv: Optional[pd.DataFrame] = None,
                               cluster_rca_sbm: Optional[pd.DataFrame] = None) -> pd.DataFrame:
    out = panel.copy()
    if cluster_pmi_louv is not None:
        comm = _build_project_to_community(cluster_pmi_louv)
        out = out.merge(comm, on='project', how='left')
        out = out.rename(columns={'community':'community_pmi_louv','description_simple':'description_pmi_louv'})
    if cluster_rca_sbm is not None:
        comm = _build_project_to_community(cluster_rca_sbm)
        out = out.merge(comm, on='project', how='left')
        out = out.rename(columns={'community':'community_rca_sbm','description_simple':'description_rca_sbm'})
    return out


def build_user_quarter_community_metrics(panel: pd.DataFrame, df_commit: pd.DataFrame, lib_to_comm: Dict[str, Any], suffix: str) -> pd.DataFrame:
    cl = commit_libs(df_commit)
    cl['commits_communities'] = cl['libs'].apply(lambda L: sorted(set(lib_to_comm.get(lib) for lib in L if lib in lib_to_comm)))
    uq = (
        cl.groupby(['user_hashed','year_quarter'], as_index=False)
          .agg(comm_lists=('commits_communities', list))
          .sort_values(['user_hashed','year_quarter'])
    )
    uq['comm_flat'] = uq['comm_lists'].apply(lambda lists: [c for row in lists for c in row])

    uniq, new = _per_user_chronological_counts(uq, 'comm_flat')
    uq[f'unique_import_communities_{suffix}'] = uniq
    uq[f'new_unique_import_communities_{suffix}'] = new
    uq[f'globally_new_import_communities_{suffix}'] = _global_novelty_over_time(uq, 'comm_flat')

    grid = panel[['user_hashed','year_quarter']].drop_duplicates()
    out = grid.merge(
        uq[['user_hashed','year_quarter',
            f'unique_import_communities_{suffix}',
            f'new_unique_import_communities_{suffix}',
            f'globally_new_import_communities_{suffix}']],
        on=['user_hashed','year_quarter'], how='left'
    )
    cols = [f'unique_import_communities_{suffix}',
            f'new_unique_import_communities_{suffix}',
            f'globally_new_import_communities_{suffix}']
    out[cols] = out[cols].fillna(0).astype('int64')
    return out


In [ ]:
# ---------------------------------------
# Cushion & year subset
# ---------------------------------------
def apply_cushion(panel: pd.DataFrame, cushion: int) -> pd.DataFrame:
    order = _quarter_order_map(panel['year_quarter'])
    out = panel.assign(_ord=panel['year_quarter'].map(order)).sort_values(['user_hashed','_ord'])
    def _drop_k(g: pd.DataFrame) -> pd.DataFrame:
        return g.iloc[cushion:]
    out = out.groupby('user_hashed', group_keys=False).apply(_drop_k).reset_index(drop=True)
    return out.drop(columns=['_ord'])


def global_subset_year(panel: pd.DataFrame, year_min_exclusive: int) -> pd.DataFrame:
    if 'year' not in panel.columns:
        panel = panel.assign(year=panel['year_quarter'].str.slice(0,4).astype('int64'))
    return panel[panel['year'] > year_min_exclusive].copy()


In [ ]:
# ---------------------------------------
# Orchestrator
# ---------------------------------------
def build_full_panel(df_raw: pd.DataFrame,
                     cluster_pmi_louv: Optional[pd.DataFrame] = None,
                     cluster_rca_sbm: Optional[pd.DataFrame] = None,
                     settings: Settings = Settings()) -> pd.DataFrame:
    commits = build_commit_level(df_raw)
    panel = build_base_panel(df_raw, commits)

    lib_metrics = build_user_quarter_library_metrics(panel, commits)
    panel = panel.merge(lib_metrics, on=['user_hashed','year_quarter'], how='left')

    pair_metrics = build_user_quarter_pair_metrics(panel, commits, settings.top_k_libs, settings.top_k_coverage)
    panel = panel.merge(pair_metrics, on=['user_hashed','year_quarter'], how='left')

    combo_metrics = build_user_quarter_commit_combo_metrics(panel, commits, settings.min_combo_size)
    panel = panel.merge(combo_metrics, on=['user_hashed','year_quarter'], how='left')

    panel = attach_project_communities(panel, cluster_pmi_louv, cluster_rca_sbm)

    if cluster_pmi_louv is not None and 'libraries' in cluster_pmi_louv.columns:
        lib2comm_pmi = _build_library_to_community_map(cluster_pmi_louv, 'libraries')
        uqc_pmi = build_user_quarter_community_metrics(panel, commits, lib2comm_pmi, 'pmi_louv')
        panel = panel.merge(uqc_pmi, on=['user_hashed','year_quarter'], how='left')

    if cluster_rca_sbm is not None and 'nodes' in cluster_rca_sbm.columns:
        lib2comm_rca = _build_library_to_community_map(cluster_rca_sbm, 'nodes')
        uqc_rca = build_user_quarter_community_metrics(panel, commits, lib2comm_rca, 'rca_sbm')
        panel = panel.merge(uqc_rca, on=['user_hashed','year_quarter'], how='left')

    if settings.cushion_quarters and settings.cushion_quarters > 0:
        panel = apply_cushion(panel, settings.cushion_quarters)

    if settings.global_year_min is not None:
        panel = global_subset_year(panel, settings.global_year_min)

    # Back-compat column aliases
    alias_map = {
        'unique_import_entries': 'unique_libs',
        'new_unique_import_entries': 'new_unique_libs',
        'globally_new_unique_import_entries': 'globally_new_libs',
        'unique_import_lists': 'unique_commit_combos',
        'new_unique_import_lists': 'new_unique_commit_combos',
        'globally_new_unique_import_lists': 'globally_new_commit_combos',
    }
    for alias, src_col in alias_map.items():
        if src_col in panel.columns and alias not in panel.columns:
            panel[alias] = panel[src_col]

    # Final column order (only include if present)
    final_cols = [
        'user_hashed','project','year_quarter','year','quarter',
        'n_commits','n_commits_multi_files','n_commits_with_import',
        'mean_ai_share',#'ai_share_window',
        'project_n_people','project_n_commits','project_n_files',
        'unique_libs','new_unique_libs','globally_new_libs',
        'unique_pairs_topk','new_unique_pairs_topk','globally_new_pairs_topk',
        'unique_commit_combos','new_unique_commit_combos','globally_new_commit_combos',
        'community_pmi_louv','description_pmi_louv',
        'community_rca_sbm','description_rca_sbm',
        'unique_import_communities_pmi_louv','new_unique_import_communities_pmi_louv','globally_new_import_communities_pmi_louv',
        'unique_import_communities_rca_sbm','new_unique_import_communities_rca_sbm','globally_new_import_communities_rca_sbm',
    ]
    existing = [c for c in final_cols if c in panel.columns]
    panel = panel[existing + [c for c in panel.columns if c not in existing]]
    return panel


In [ ]:
# ---------------------------------------
# Notebook runner helper
# ---------------------------------------
def run_pipeline_in_notebook(raw_csv: str,
                             out_csv: str = 'panel.csv',
                             cluster_pmi_louv_csv: Optional[str] = None,
                             cluster_rca_sbm_csv: Optional[str] = None,
                             settings: Settings = Settings()) -> pd.DataFrame:
    df = pd.read_csv(raw_csv)
    cluster_pmi = pd.read_csv(cluster_pmi_louv_csv) if cluster_pmi_louv_csv else None
    cluster_rca = pd.read_csv(cluster_rca_sbm_csv) if cluster_rca_sbm_csv else None
    panel = build_full_panel(df, cluster_pmi, cluster_rca, settings=settings)
    panel.to_csv(out_csv, index=False)
    return panel


In [ ]:
run_pipeline_in_notebook("./final_data/raw_data_encrypted.csv.zip",cluster_pmi_louv_csv="./final_data/project_by_task_from_library_uni_network.csv",cluster_rca_sbm_csv="./final_data/project_by_project_library_bipartite_rca_sbm.csv")

### Optional: quick smoke test (synthetic data)
Run the cell below to verify shapes on toy data. Safe to skip.


In [ ]:
# Synthetic minimal example (uncomment to run)
# data = [
#     dict(user_hashed='u1', project='pA', commit_id='c1', file_name='f1.py', year_quarter='2021-Q1', imports_commit="['pandas','numpy']", ai_share=0.3),
#     dict(user_hashed='u1', project='pA', commit_id='c2', file_name='f2.py', year_quarter='2021-Q2', imports_commit="['pandas','scipy']", ai_share=0.5),
#     dict(user_hashed='u2', project='pB', commit_id='c3', file_name='g1.py', year_quarter='2021-Q2', imports_commit="['numpy']", ai_share=0.4),
# ]
# df = pd.DataFrame(data)
# panel = build_full_panel(df, settings=Settings(cushion_quarters=0, global_year_min=None, top_k_libs=2, min_combo_size=2))
# panel.head()
